In [1]:
import pandas as pd
import torchtext
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, Vocab
import numpy as np
import html
import pickle

In [2]:
V = np.load("./2022201068_Assignment3/svd_model/ipca_matrix.npy", allow_pickle=True)
with open('./2022201068_Assignment3/svd_model/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [3]:
U_df = pd.DataFrame(V, index=vocab.get_stoi().keys())
U_df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
zuzu,0.515255,0.723150,-3.228056e-01,-2.660664e-01,-5.476174e-02,-1.999388e-02,-1.859012e-02,2.113803e-02,-3.342088e-02,3.697194e-02,...,6.210430e-04,-0.000497,0.000153,0.000215,3.479734e-04,-1.121633e-03,0.001664,-0.002940,0.000195,0.000049
zelda,0.488546,-0.585208,-5.709124e-01,1.112737e-01,-2.122366e-01,2.353183e-02,-9.840652e-02,-1.162373e-01,-6.111807e-02,4.196591e-02,...,6.959570e-04,0.000698,-0.000860,-0.001023,-1.095548e-03,1.771450e-03,0.000697,0.000436,-0.000728,-0.001135
zefferelli,0.444914,-0.253860,5.869238e-01,-5.017357e-01,-1.705553e-01,-6.843296e-03,8.283414e-02,8.182239e-02,-1.930349e-01,-1.666029e-01,...,-6.416084e-04,0.000102,-0.000626,-0.004185,-3.272435e-04,1.549603e-03,0.001207,0.001742,0.002351,0.002367
zahn,0.313481,0.187275,3.801039e-01,7.447551e-01,-3.602367e-01,-1.368737e-01,-1.317053e-03,-8.743688e-02,-1.509413e-02,3.135979e-02,...,1.822010e-03,-0.000631,-0.000590,0.000087,-8.518810e-04,1.386791e-03,0.000727,0.000008,-0.003404,-0.002330
yum,0.180950,0.023451,1.395238e-01,1.521292e-01,3.151889e-01,8.336518e-01,-3.360960e-01,4.537947e-02,-1.340370e-02,2.139150e-02,...,1.317420e-04,0.001682,-0.002239,-0.000296,5.537863e-04,3.076109e-03,-0.000385,0.001280,0.000488,-0.000287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fear,0.000004,-0.000001,2.781411e-07,5.053212e-06,9.534522e-06,5.465845e-06,-3.237676e-06,-2.077986e-06,-9.905641e-06,-2.217648e-06,...,4.266536e-07,0.000007,0.000019,-0.000013,-1.028026e-05,1.947821e-05,0.000007,-0.000013,0.000005,-0.000001
loneliness,0.000007,0.000002,-4.387408e-06,3.654438e-06,-3.709171e-06,1.559146e-06,-4.448123e-06,-3.999067e-06,7.623452e-07,7.611853e-07,...,-1.130022e-05,0.000005,-0.000021,-0.000016,-6.181863e-06,6.650670e-06,0.000037,-0.000028,-0.000021,0.000003
ambitious,0.000004,-0.000005,2.219796e-06,-3.845981e-07,5.429174e-06,-4.894858e-08,2.458608e-08,4.512627e-06,-4.554397e-06,-3.066261e-06,...,7.411026e-07,0.000025,0.000005,-0.000064,-1.351181e-04,-1.743987e-04,-0.000025,0.000025,-0.000014,-0.000003
vignette,0.000006,0.000004,-2.732768e-06,2.559182e-06,-1.899964e-07,5.271248e-06,-9.131289e-07,3.607374e-07,-8.145767e-06,3.671824e-06,...,-1.712351e-06,-0.000005,-0.000015,-0.000009,9.655806e-07,-8.812383e-07,0.000028,-0.000030,-0.000008,0.000006


In [4]:
from sklearn.metrics.pairwise import cosine_similarity

def getTopSimilarWords(word):
    word_index = vocab[word]
    word_vector = V[word_index]


    similarities = []
    for i in range(V.shape[0]):
        simi = cosine_similarity(V[i].reshape(1, -1), [word_vector])
        similarities.extend(simi.flatten())

    k = 10
    top_k_similar_words = np.argsort(similarities)[-k:]

    index_to_words = vocab.get_itos()

    return list(reversed([index_to_words[idx] for idx in top_k_similar_words])), list(reversed([similarities[idx] for idx in top_k_similar_words]))



In [10]:
getTopSimilarWords("book")

(['book',
  'comic',
  'pillow',
  'candle',
  'hardcover',
  'bell',
  'krakauer',
  'mormon',
  'massie',
  'bestselling'],
 [0.9999999999999997,
  0.9756183927843077,
  0.9754146828244334,
  0.9641468076449972,
  0.9590255156597187,
  0.9395908168822863,
  0.9350034775371506,
  0.9341182469107848,
  0.9304497885029396,
  0.9055292280061048])

In [30]:
getTopSimilarWords("camera")

(['camera',
  'angles',
  'tilted',
  'close-ups',
  'shots',
  'zooms',
  'lens',
  'movements',
  'positioning',
  'cameras'],
 [1.0000000000000002,
  0.752909476167504,
  0.5617427320021571,
  0.5287126666937313,
  0.508121636979776,
  0.5009796416898574,
  0.4840435495383279,
  0.478028396655714,
  0.4655174533958218,
  0.46152816265683577])

In [31]:
getTopSimilarWords("titanic")

(['titanic',
  'rms',
  'iceberg',
  '1912',
  'sinking',
  'ship',
  'unsinkable',
  'sank',
  'ismay',
  'carpathia'],
 [1.0000000000000002,
  0.7055050115825704,
  0.6047698602963449,
  0.5919187138933221,
  0.5647374918952452,
  0.536203400366257,
  0.5256652717478577,
  0.487277602375207,
  0.4816137386697942,
  0.47812195890377934])

In [32]:
getTopSimilarWords("fun")

(['fun',
  'entertaining',
  'romp',
  'enjoyable',
  'light-hearted',
  'energizing',
  'banter',
  'colorful',
  'entertainment',
  'lighthearted'],
 [1.0,
  0.46222288051408916,
  0.44606535785263474,
  0.4085711152776538,
  0.38739053551730745,
  0.36134573597152375,
  0.35962724292559856,
  0.34985910102241,
  0.3407499133483285,
  0.33032024546485117])

In [33]:
getTopSimilarWords("definitely")

(['definitely',
  'faint-hearted',
  'worth',
  'owning',
  'well-worth',
  'deffinately',
  'defintely',
  'recommend',
  'definitly',
  'recommened'],
 [1.0,
  0.509364393875435,
  0.5037084709835571,
  0.4491195766539429,
  0.445851399740761,
  0.4395179666257254,
  0.4355857860747184,
  0.4350325176564546,
  0.4288286519041118,
  0.4216144296774996])